前面学习的几个索引类型为了实现向量搜索，都需要将向量存储到 Faiss 中，当向量的数量较多时就会占用更多的内存。 这也影响了 Faiss 的应用。所以，为了减少内存的占用，我们就需要会存储的向量进行重新编码、压缩，使其占用更少的存储内存，从而能够容纳更多的向量。

量化技术(Product Quantization)可以使用较低精度的表示来近似向量数据，从而降低内存需求而又不牺牲准确性。 这对于大规模向量相似性搜索应用程序特别有用。

In [ ]:
import os

import faiss
import numpy as np
import time


np.random.seed(0)
data = np.random.rand(1000000, 256)
ids = np.arange(0, 1000000)
query_vector = np.random.rand(1, 256)


def test01():

    index = faiss.IndexFlatL2(256)
    index = faiss.IndexIDMap(index)
    # 添加向量
    index.add_with_ids(data, ids)
    # 搜索向量
    s = time.time()
    D, I = index.search(query_vector, k=2)
    print('time:', time.time() - s)
    print(D, I)

    # faiss.write_index(index, 'flat.faiss')
    # print(os.stat('flat.faiss').st_size)


def test03():

    # 第一个参数：量化参数
    # 第二个参数：向量维度
    # 第三个参数：质心数量
    # 第四个参数：子空间数量（或称为段数）, 较大的值意味着将原始向量空间划分为更多的子空间进行量化，有助于减少量化误差，因为每个子空间都将被更精细地量化。
    # 第五个参数：量化码本中码字的位数，每个段聚类的数量（8位256），决定了每个量化码字的精度，位数越多，每个码字能够表示的信息就越多，量化误差就越小。
    quantizer = faiss.IndexFlatL2(256)
    index = faiss.IndexIVFPQ(quantizer, 256, 100, 128, 10)
    index.nprobe = 10
    index.train(data)
    index.add(data)
    # index.add_with_ids(data, ids)
    # 搜索向量
    start = time.time()
    D, I = index.search(query_vector, k=2)
    print('time:', time.time() - start)
    print(D, I)

    # faiss.write_index(index, 'ivfpq.faiss')
    # print(os.stat('ivfpq.faiss').st_size)


if __name__ == '__main__':
    test01()
    test03()

# The following is from faiss github

In [3]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [4]:

nlist = 100
m = 8                             # number of subquantizers
k = 4
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                    # 8 specifies that each sub-vector is encoded as 8 bits
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
index.nprobe = 10              # make comparable with experiment above
D, I = index.search(xq, k)     # search
print(D[-5:])
print(I[-5:])

[[   0   78  608  159]
 [   1 1063  555  380]
 [   2  179  304  134]
 [   3  484  265   64]
 [   4  288  827  531]]
[[1.597085  5.9489446 6.531045  6.576322 ]
 [1.3748417 6.0429764 6.437569  6.4838734]
 [1.7478867 6.1723785 6.18528   6.637947 ]
 [1.9041667 6.4681654 6.7150736 6.8242025]
 [1.6288457 5.7781396 6.384511  6.436339 ]]
[[ 9853  8746  9900 10437]
 [10494 10507 10913 10260]
 [10719 11291 10424  9995]
 [10122 10005 11578 10125]
 [ 9229 10304  9878 10152]]


In [5]:
print(D[-5:])
print(I[-5:])

[[6.092051  6.1051054 6.1229076 6.175897 ]
 [4.4511356 4.6641293 4.7500663 4.835972 ]
 [5.320959  5.686492  6.013778  6.0157094]
 [5.2807274 6.035577  6.191619  6.2410173]
 [4.917555  5.2247696 5.325024  5.629692 ]]
[[ 9853  8746  9900 10437]
 [10494 10507 10913 10260]
 [10719 11291 10424  9995]
 [10122 10005 11578 10125]
 [ 9229 10304  9878 10152]]
